# Code

In [27]:
import pandas as pd
from pathlib import Path
import re, glob
from datetime import datetime

In [28]:
folder_path = Path(r'./data/')
file = list(folder_path.rglob('*Fleetpin*.csv'))
df = pd.read_csv(file[0])
selector = ['Name', 'Odo / Hours / Hubo', 'Next Service Due', 'RUC Due', 'WoF Due', 'Rego Due']
df = df[selector]

In [29]:
df

,Name,Odo / Hours / Hubo,Next Service Due,RUC Due,WoF Due,Rego Due
0,101 - JLM368 - Hiace,371971 km,375000 km,388652\nkm,16/11/2023,24/11/2023
1,102 - LGH14 - Hiace,421977 km,435000 km,436000\nkm,30/04/2024,19/03/2024
2,103 - LPB86 - Hiace,293533 km,294500 km,301948\nkm,10/08/2024,15/08/2024
3,104 - LPB87 - Hiace,271762 km,283600 km,288404\nkm,13/09/2023,15/08/2024
4,105 - LPK832 - Hiace,297725 km,314404 km,308892\nkm,23/08/2024,26/08/2024
5,106 - KBW622 - KK Hiace (small chiller),251315 km,260000 km,268000\nkm,30/11/2023,22/01/2024
6,107 - LZN203 - Transit,195070 km,200000 km,204390\nkm,28/08/2024,26/02/2024
7,110 - PHK242 - Sprinter,103319 km\n\n\n/ 103245 km,120000 km,111000\nkm,13/09/2023,28/06/2024
8,111 - NLS732 - Sprinter,218029 km\n\n\n/ 173971 km,225575 km,180002\nkm,29/07/2023,28/04/2024
9,112 - NNG481 - Sprinter,220859 km\n\n\n/ 223624 km,248459 km,236167\nkm,25/11/2023,31/05/2024


In [30]:
df.dtypes

Name                  object
Odo / Hours / Hubo    object
Next Service Due      object
RUC Due               object
WoF Due               object
Rego Due              object
dtype: object

In [31]:
# convert 'WoF Due' and 'Rego Due' data from string to datetime
df['WoF Due'] = pd.to_datetime(df['WoF Due'], dayfirst=True)
df['Rego Due'] = pd.to_datetime(df['Rego Due'], dayfirst=True)

# get today's date for comparison
today = datetime.now().date()

# date calculations - subtract 'WoF Due' date from today's date to get days remaining for WOF and Rego renewals
df['WOF Renew'] = (df['WoF Due'] - pd.Timestamp(today)).dt.days
df['Rego Renew'] = (df['Rego Due'] - pd.Timestamp(today)).dt.days

In [32]:
# cleaning data by removing characters from numbers using regex
# df['Odo / Hours / Hubo'] = pd.Series(df['Odo / Hours / Hubo']).str.findall(r'\d+')
df['Odo / Hours / Hubo'] = (df['Odo / Hours / Hubo']).str.findall(r'\d+')

# get rid of list-like structure and to strings where the elements are joined by comma
df['Odo / Hours / Hubo'] = [','.join(map(str, l)) for l in df['Odo / Hours / Hubo']]

# split values seprated by comma
df['ODO'] = df['Odo / Hours / Hubo'].apply(lambda x: x.split(',')[0])
df['Hubometer'] = df['Odo / Hours / Hubo'].apply(lambda x: x.split(',')[-1])

# get rid of 'km' string at the end
df['Next Service Due'] = (df['Next Service Due']).str.strip('km')

# use regex to fetch only the numbers
df['RUC Due'] = pd.Series(df['RUC Due']).str.findall(r'\b\d+\b')
# get rid of list-like structure and to strings where the elements are joined by comma
df['RUC Due']  = [','.join(map(str, l)) for l in df['RUC Due'] ]

# convert to integer
df['Next Service Due'] = df['Next Service Due'].apply(lambda x: int(x))
df['RUC Due'] = df['RUC Due'].apply(lambda x: int(x))
df['ODO'] = df['ODO'].apply(lambda x: int(x))
df['Hubometer'] = df['Hubometer'].apply(lambda x: int(x))

# calculations
df['RUC Remaining'] = df['RUC Due'] - df['Hubometer']
df['Next Service In'] = df['Next Service Due'] - df['ODO']


In [33]:
# change date format
df['WoF Due*'] = df['WoF Due'].apply(lambda x: x.strftime('%d-%m-%Y'))
df['Rego Due*'] = df['Rego Due'].apply(lambda x: x.strftime('%d-%m-%Y'))

In [34]:
# re-ordering column
df = df[['Name','ODO','Hubometer','Next Service Due','RUC Due','Rego Due*','WoF Due*','Next Service In','WOF Renew','Rego Renew','RUC Remaining']]


In [35]:
df.head()

,Name,ODO,Hubometer,Next Service Due,RUC Due,Rego Due*,WoF Due*,Next Service In,WOF Renew,Rego Renew,RUC Remaining
0,101 - JLM368 - Hiace,371971,371971,375000,388652,24-11-2023,16-11-2023,3029,81,89,16681
1,102 - LGH14 - Hiace,421977,421977,435000,436000,19-03-2024,30-04-2024,13023,247,205,14023
2,103 - LPB86 - Hiace,293533,293533,294500,301948,15-08-2024,10-08-2024,967,349,354,8415
3,104 - LPB87 - Hiace,271762,271762,283600,288404,15-08-2024,13-09-2023,11838,17,354,16642
4,105 - LPK832 - Hiace,297725,297725,314404,308892,26-08-2024,23-08-2024,16679,362,365,11167


In [36]:
df.dtypes

Name                object
ODO                  int64
Hubometer            int64
Next Service Due     int64
RUC Due              int64
Rego Due*           object
WoF Due*            object
Next Service In      int64
WOF Renew            int64
Rego Renew           int64
RUC Remaining        int64
dtype: object

# Styling

In [37]:
df_styled = df.style\
    .applymap(lambda x: 'background-color: %s' % '#ff9966' if x < 0 else ('background-color: %s' % '#99ff99' if 0 <= x <= 5000 else 'background-color: %s' % 'none'), subset=['Next Service In'])\
    .applymap(lambda x: 'background-color: %s' % '#ff9966' if x < 0 else 'background-color: %s' % 'none', subset=['RUC Remaining'])\
    .highlight_between(subset='RUC Remaining',left=0,right=7000, color='#ff9966')\
    .applymap(lambda x: 'background-color: %s' % '#ff9966' if x < 0 else ('background-color: %s' % '#99ff99' if 0 <= x <= 30 else 'background-color: %s' % 'none'), subset=['WOF Renew'])\
    .applymap(lambda x: 'background-color: %s' % '#ff9966' if x < 0 else ('background-color: %s' % '#99ff99' if 0 <= x <= 30 else 'background-color: %s' % 'none'), subset=['Rego Renew'])\
    .format({'RUC Remaining': '{0:,.0f} km', 'Next Service In': '{0:,.0f} km', 'ODO': '{0:,.0f} km', 'Hubometer': '{0:,.0f} km', 'Next Service Due': '{0:,.0f} km', 'RUC Due': '{0:,.0f} km',
             'WOF Renew': '{} Days', 'Rego Renew': '{} Days'})



# Export to excel

In [38]:
folder_path_report = Path(r'./report/')
file_path = folder_path_report / 'Report.xlsx'
df_styled.to_excel(file_path, engine='openpyxl', index=False)

# Report

In [39]:
df_styled

,Name,ODO,Hubometer,Next Service Due,RUC Due,Rego Due*,WoF Due*,Next Service In,WOF Renew,Rego Renew,RUC Remaining
0,101 - JLM368 - Hiace,"371,971 km","371,971 km","375,000 km","388,652 km",24-11-2023,16-11-2023,"3,029 km",81 Days,89 Days,"16,681 km"
1,102 - LGH14 - Hiace,"421,977 km","421,977 km","435,000 km","436,000 km",19-03-2024,30-04-2024,"13,023 km",247 Days,205 Days,"14,023 km"
2,103 - LPB86 - Hiace,"293,533 km","293,533 km","294,500 km","301,948 km",15-08-2024,10-08-2024,967 km,349 Days,354 Days,"8,415 km"
3,104 - LPB87 - Hiace,"271,762 km","271,762 km","283,600 km","288,404 km",15-08-2024,13-09-2023,"11,838 km",17 Days,354 Days,"16,642 km"
4,105 - LPK832 - Hiace,"297,725 km","297,725 km","314,404 km","308,892 km",26-08-2024,23-08-2024,"16,679 km",362 Days,365 Days,"11,167 km"
5,106 - KBW622 - KK Hiace (small chiller),"251,315 km","251,315 km","260,000 km","268,000 km",22-01-2024,30-11-2023,"8,685 km",95 Days,148 Days,"16,685 km"
6,107 - LZN203 - Transit,"195,070 km","195,070 km","200,000 km","204,390 km",26-02-2024,28-08-2024,"4,930 km",367 Days,183 Days,"9,320 km"
7,110 - PHK242 - Sprinter,"103,319 km","103,245 km","120,000 km","111,000 km",28-06-2024,13-09-2023,"16,681 km",17 Days,306 Days,"7,755 km"
8,111 - NLS732 - Sprinter,"218,029 km","173,971 km","225,575 km","180,002 km",28-04-2024,29-07-2023,"7,546 km",-29 Days,245 Days,"6,031 km"
9,112 - NNG481 - Sprinter,"220,859 km","223,624 km","248,459 km","236,167 km",31-05-2024,25-11-2023,"27,600 km",90 Days,278 Days,"12,543 km"
